In [1]:
from pybaseball import statcast
from pybaseball import retrosheet

In [63]:
data = statcast(start_dt = '2015-01-01', end_dt='2015-05-31')

This is a large query, it may take a moment to complete
Completed sub-query from 2015-01-01 to 2015-05-31


In [80]:
data.columns

Index(['index', 'pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
     

In [81]:
data['estimated_woba_using_speedangle'].describe()

count    7368.000000
mean        0.354850
std         0.363323
min         0.000000
25%         0.068000
50%         0.254000
75%         0.542000
max         2.025000
Name: estimated_woba_using_speedangle, dtype: float64

In [96]:
# create pa_id
data['pa_id'] = data.groupby(by=['at_bat_number', 'game_date', 'batter', 'pitcher']).ngroup()


# TODO specify this better by adding in day/night or some other identifier. Currently using pitcher, but causes issues when a pitcher is taken out mid-PA (maybe I want to classify that as a new pa though?). Without pitcher, double-headers are an issue. If the same batter hits leadoff both games, both of those leadoff PAs are going to be counted as the same pa_id

In [97]:
# checking how many duplicate plate appearances there are (double headers where the same hitter has the same at_bat_number agains the same pitcher, very unlikely)
data[data['pitch_number']==1]['pa_id'].value_counts().value_counts()

1    10604
Name: pa_id, dtype: int64

In [ ]:
data[

In [84]:
data[data['pitch_number']==1]['pitcher'].value_counts().describe()

count    401.000000
mean      26.443890
std       17.083397
min        1.000000
25%       13.000000
50%       20.000000
75%       47.000000
max       64.000000
Name: pitcher, dtype: float64

In [85]:
data['game_date'].describe()

count                   40000
unique                     10
top       2015-05-31 00:00:00
freq                     4636
first     2015-05-22 00:00:00
last      2015-05-31 00:00:00
Name: game_date, dtype: object

In [86]:
# create xwOBA using est_wOBA if in play or wOBA_value if not in play
data['xwOBA'] = data['woba_value']

data.loc[data.estimated_woba_using_speedangle.notnull(), "xwOBA"] = data.loc[data.estimated_woba_using_speedangle.notnull(), 'estimated_woba_using_speedangle']

In [87]:
data.loc[data.estimated_woba_using_speedangle.notnull(), "xwOBA"]

0        0.384
5        0.040
9        0.211
15       0.242
18       0.068
         ...  
39974    0.488
39987    0.023
39994    0.006
39996    0.044
39999    1.999
Name: xwOBA, Length: 7368, dtype: float64

In [88]:
# get pa_xwOBA values showing the resulting xwOBA of each plate appearance
gb = data.groupby(by=['pa_id'])['pa_id', 'xwOBA'].mean()

data = data.join(gb, how='left', on='pa_id', rsuffix='gb')
data['pa_xwOBA'] = data['xwOBAgb']
data.drop(columns=['pa_idgb', 'xwOBAgb'], inplace=True)


In [89]:

data.describe()

,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,...,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,pa_id,xwOBA,pa_xwOBA
count,40000.00000,39997.000000,39310.000000,39310.000000,40000.000000,40000.000000,0.0,0.0,0.0,0.0,...,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,10618.000000,39997.000000
mean,19999.50000,88.603763,-0.859654,6.035350,489515.639900,495407.626225,NaN,NaN,NaN,NaN,...,2.079625,2.065650,2.100600,2.079675,2.086625,2.065700,2.100600,5313.708400,0.304095,0.297304
std,11547.14972,6.105128,1.791526,0.508735,84951.334849,78926.695515,NaN,NaN,NaN,NaN,...,2.526954,2.429308,2.485545,2.526922,2.386328,2.429276,2.485545,3086.267071,0.360519,0.355822
min,0.00000,46.500000,-15.880000,0.240000,112526.000000,112526.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9999.75000,84.600000,-2.090000,5.770000,451594.000000,451788.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2629.750000,0.004000,0.000000
50%,19999.50000,89.900000,-1.410000,6.070000,488862.000000,502026.000000,NaN,NaN,NaN,NaN,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5327.000000,0.152000,0.132000
75%,29999.25000,93.200000,0.050000,6.340000,543543.000000,543606.000000,NaN,NaN,NaN,NaN,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,7999.000000,0.548000,0.578000
max,39999.00000,103.100000,13.840000,8.860000,648717.000000,628452.000000,NaN,NaN,NaN,NaN,...,15.000000,15.000000,15.000000,15.000000,14.000000,15.000000,15.000000,10608.000000,2.025000,2.025000


In [90]:
data[['pa_id', 'xwOBA']].head()

,pa_id,xwOBA
0,9789,0.384
1,9789,NaN
2,9789,NaN
3,9789,NaN
4,9789,NaN


In [91]:
data[['sv_id','pa_id','pa_xwOBA','xwOBA','at_bat_number', 'pitch_number','pitcher','batter','game_date','strikes','balls','pitch_type', 'batter', 'pitcher','stand', 'description', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom', 'babip_value']].sort_values(by='pa_id').tail(25)

,sv_id,pa_id,pa_xwOBA,xwOBA,at_bat_number,pitch_number,pitcher,batter,game_date,strikes,...,pitch_type,batter,pitcher,stand,description,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value
4128,150531_184606,10602,0.148,0.148,140.0,6.0,573064.0,467070.0,2015-05-31,2.0,...,FF,467070.0,573064.0,R,hit_into_play,0.149,0.148,0.0,1.0,0.0
4129,150531_184548,10602,0.148,NaN,140.0,5.0,573064.0,467070.0,2015-05-31,2.0,...,SL,467070.0,573064.0,R,ball,NaN,NaN,NaN,NaN,NaN
4130,150531_184532,10602,0.148,NaN,140.0,4.0,573064.0,467070.0,2015-05-31,2.0,...,FF,467070.0,573064.0,R,ball,NaN,NaN,NaN,NaN,NaN
4132,150531_184502,10602,0.148,NaN,140.0,2.0,573064.0,467070.0,2015-05-31,1.0,...,FF,467070.0,573064.0,R,ball,NaN,NaN,NaN,NaN,NaN
4133,150531_184443,10602,0.148,NaN,140.0,1.0,573064.0,467070.0,2015-05-31,0.0,...,SL,467070.0,573064.0,R,foul_tip,NaN,NaN,NaN,NaN,NaN
4126,150531_184937,10603,0.224,0.224,141.0,2.0,490063.0,457574.0,2015-05-31,0.0,...,FT,457574.0,490063.0,R,hit_into_play,0.249,0.224,0.0,1.0,0.0
4127,150531_184907,10603,0.224,NaN,141.0,1.0,490063.0,457574.0,2015-05-31,0.0,...,FT,457574.0,490063.0,R,ball,NaN,NaN,NaN,NaN,NaN
4123,150531_185120,10604,0.174,0.174,142.0,3.0,490063.0,605113.0,2015-05-31,2.0,...,SL,605113.0,490063.0,R,hit_into_play_no_out,0.169,0.174,0.9,1.0,1.0
4125,150531_185025,10604,0.174,NaN,142.0,1.0,490063.0,605113.0,2015-05-31,0.0,...,FT,605113.0,490063.0,R,swinging_strike,NaN,NaN,NaN,NaN,NaN
4124,150531_185053,10604,0.174,NaN,142.0,2.0,490063.0,605113.0,2015-05-31,1.0,...,FT,605113.0,490063.0,R,called_strike,NaN,NaN,NaN,NaN,NaN


In [11]:
data.of_fielding_alignment.unique()

array(['Standard', 'Strategic', nan, 'Extreme outfield shift'],
      dtype=object)

In [76]:
.head(20)

0     150531_155632
1     150531_155603
2     150531_155543
3     150531_155518
4     150531_155502
5     150531_155421
6     150531_155402
7     150531_155341
8     150531_155323
9     150531_155247
10    150531_155221
11    150531_155158
12    150531_155133
13    150531_155115
14    150531_155058
15    150531_154755
16    150531_154736
17    150531_154721
18    150531_154646
19    150531_154623
Name: sv_id, dtype: object

In [16]:
data.head(20)

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,pa_id,xwOBA,pa_xwOBA
0,256,FF,2015-05-04,97.1,-1.22,5.88,Neftali Feliz,474892.0,491703.0,strikeout,...,2.0,2.0,1.0,1.0,2.0,Standard,Standard,3671,0.000,0.000
1,258,SL,2015-05-04,83.2,-1.80,5.70,Neftali Feliz,474892.0,491703.0,NaN,...,2.0,2.0,1.0,1.0,2.0,Standard,Standard,3671,NaN,0.000
2,268,SL,2015-05-04,83.0,-1.49,5.67,Neftali Feliz,474892.0,491703.0,NaN,...,2.0,2.0,1.0,1.0,2.0,Standard,Standard,3671,NaN,0.000
3,276,SL,2015-05-04,84.6,-1.55,5.58,Neftali Feliz,474892.0,491703.0,NaN,...,2.0,2.0,1.0,1.0,2.0,Standard,Strategic,3671,NaN,0.000
4,278,FF,2015-05-04,95.8,-1.27,5.99,Neftali Feliz,474892.0,491703.0,NaN,...,2.0,2.0,1.0,1.0,2.0,Standard,Strategic,3671,NaN,0.000
5,289,SL,2015-05-04,84.2,-1.46,5.75,Neftali Feliz,594828.0,491703.0,field_out,...,2.0,2.0,1.0,1.0,2.0,Standard,Strategic,3626,0.090,0.090
6,298,SL,2015-05-04,83.7,-1.59,5.96,Neftali Feliz,594828.0,491703.0,NaN,...,2.0,2.0,1.0,1.0,2.0,Standard,Strategic,3626,NaN,0.090
7,300,FF,2015-05-04,94.1,-1.43,5.95,Neftali Feliz,543807.0,491703.0,strikeout,...,2.0,2.0,1.0,1.0,2.0,Standard,Standard,3569,0.000,0.000
8,308,SL,2015-05-04,81.4,-1.36,5.76,Neftali Feliz,543807.0,491703.0,NaN,...,2.0,2.0,1.0,1.0,2.0,Standard,Standard,3569,NaN,0.000
9,316,SL,2015-05-04,83.1,-1.59,5.89,Neftali Feliz,543807.0,491703.0,NaN,...,2.0,2.0,1.0,1.0,2.0,Standard,Strategic,3569,NaN,0.000


In [13]:
list(retrosheet.season_game_logs(2019).columns)

['date',
 'game_num',
 'day_of_week',
 'visiting_team',
 'visiting_team_league',
 'visiting_game_num',
 'home_team',
 'home_team_league',
 'home_team_game_num',
 'visiting_score',
 'home_score',
 'num_outs',
 'day_night',
 'completion_info',
 'forfeit_info',
 'protest_info',
 'park_id',
 'attendance',
 'time_of_game_minutes',
 'visiting_line_score',
 'home_line_score',
 'visiting_abs',
 'visiting_hits',
 'visiting_doubles',
 'visiting_triples',
 'visiting_homeruns',
 'visiting_rbi',
 'visiting_sac_hits',
 'visiting_sac_flies',
 'visiting_hbp',
 'visiting_bb',
 'visiting_iw',
 'visiting_k',
 'visiting_sb',
 'visiting_cs',
 'visiting_gdp',
 'visiting_ci',
 'visiting_lob',
 'visiting_pitchers_used',
 'visiting_individual_er',
 'visiting_er',
 'visiting__wp',
 'visiting_balks',
 'visiting_po',
 'visiting_assists',
 'visiting_errors',
 'visiting_pb',
 'visiting_dp',
 'visiting_tp',
 'home_abs',
 'home_hits',
 'home_doubles',
 'home_triples',
 'home_homeruns',
 'home_rbi',
 'home_sac_hits',
